In [1]:
import pandas
import numpy as np
import gym
import tensorflow as tf
from collections import deque, Counter
import tensorflow as tf


ModuleNotFoundError: No module named 'gym'

In [ ]:
env = gym.make("SpaceInvaders-v0")
n_outputs = env.action_space.n
print(n_outputs)
print(env.env.get_action_meanings())
observation = env.reset()
import tensorflow as tf
import matplotlib.pyplot as plt
for i in range(22):
    if i > 20:
        plt.imshow(observation)
        plt.show()
        observation, _, _, _ = env.step(1)